In [49]:
#1.a

from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = DiscreteBayesianNetwork([('O', 'H'), ('O', 'W'), ('H', 'R'), ('W', 'R'), ('H', 'E'), ('R', 'C')])

cpd_o = TabularCPD(variable='O', variable_card=2, values=[[0.3], [0.7]])

cpd_h = TabularCPD(variable='H', variable_card=2, values=[[0.9, 0.2],  [0.1, 0.8]], evidence=['O'], evidence_card=[2])

cpd_w = TabularCPD(variable='W', variable_card=2, values=[[0.1, 0.6],[0.9, 0.4]],evidence=['O'], evidence_card=[2])

cpd_r = TabularCPD(variable='R', variable_card=2,values=[[0.6, 0.9, 0.3, 0.5], [0.4, 0.1, 0.7, 0.5]], evidence=['H', 'W'], evidence_card=[2, 2])

cpd_e = TabularCPD(variable='E', variable_card=2,values=[[0.8, 0.2],[0.2, 0.8]],evidence=['H'], evidence_card=[2])

cpd_c = TabularCPD(variable='C', variable_card=2,values=[[0.85, 0.4], [0.15, 0.6]], evidence=['R'], evidence_card=[2])

model.add_cpds(cpd_o, cpd_h, cpd_w, cpd_r, cpd_e, cpd_c)

model.check_model()

True

In [50]:
#1.b

infer = VariableElimination(model)

q1 = infer.query(variables=['H'], evidence={'C': 0})
q2 = infer.query(variables=['E'], evidence={'C': 0})
q_map = infer.query(variables=['H', 'W'], evidence={'C': 0})
print(q_map)

+------+------+------------+
| H    | W    |   phi(H,W) |
+======+======+============+
| H(0) | W(0) |     0.1138 |
+------+------+------------+
| H(0) | W(1) |     0.3684 |
+------+------+------------+
| H(1) | W(0) |     0.2776 |
+------+------+------------+
| H(1) | W(1) |     0.2401 |
+------+------+------------+


In [51]:
# 2.a

import numpy as np
from hmmlearn import hmm

model = hmm.CategoricalHMM(n_components=3, n_iter=100)

model.startprob=np.array([0.4, 0.3, 0.3])

model.transmat =np.array(
    [[0.6, 0.3, 0.1],
     [0.2, 0.7, 0.1],
     [0.3, 0.2, 0.5]])

model.emissionprob= np.array(
    [[0.1, 0.7, 0.2],
     [0.05, 0.25, 0.7],
     [0.8, 0.15, 0.05]])

In [52]:
# 2.b

x = np.array([[1], [2], [0]])
log_prob = model.score(x)

prob_forward = np.exp(log_prob)

print(log_prob)
print(prob_forward)

-3.8576650369552503
0.021117250000000008


In [53]:
#2.c

log_prob_viterbi, state_sequence = model.decode(X)

states_map = {0: 'Walking', 1: 'Running', 2: 'Resting'}
decoded_states = [states_map[s] for s in state_sequence]

print(decoded_states)

['Walking', 'Running', 'Resting']


In [54]:
simulations = 10000
matches = 0
target_sequence = np.array([1, 2, 0])

for i in range(simulations):
    X_sim, i = model.sample(3)
    if np.array_equal(X_sim.flatten(), target_sequence):
        matches += 1

empirical_prob = matches/simulations

print(empirical_prob)
print(prob_forward)
print(abs(empirical_prob - prob_forward))

0.0208
0.021117250000000008
0.0003172500000000085
